In [ ]:
import glob
import numpy as np
import pandas as pd
import os
import tqdm
import cleese_stim as cleese
from cleese_stim.engines import FaceWarp
from feat import Detector
import matplotlib.pyplot as plt
from PIL import Image
from imageio.v2 import imread
import pingouin as pg
import cv2
from scipy.spatial import distance
from scipy import stats

# Generate randomized stimuli using CLEESE

In [ ]:
img_file = "./images/londonset_scaled/070.jpg"
config_file = "./configs/mediapipe.toml"

deformed = cleese.generate_stimuli(FaceWarp, img_file, config_file)

# Pair stimuli randomly

In [ ]:
STIM_DIR = './outputs/londonset_output/070'

stimuli = glob.glob(STIM_DIR + '/*.jpg')
stimuli.remove(STIM_DIR + os.sep + '070.jpg')
np.random.shuffle(stimuli)

# create random pairs of simuli by splitting list of stims in half and zipping them together
trials = list(zip(stimuli[:int(len(stimuli)/2)], stimuli[int(len(stimuli)/2):]))

# Detect AUs in stimuli

In [ ]:
# svm returns a binary label for each AU
# detector = Detector(au_model='svm')

# xgb returns continuous-valued detection probabilities
detector = Detector(au_model='xgb')

In [ ]:
all_dfs = []

for trial in tqdm.tqdm(trials):
    stim1 = trial[0]
    stim2 = trial[1]
    
    stim1_au_data = detector.detect_image(stim1).aus
    stim2_au_data = detector.detect_image(stim2).aus

    stim1_dfm = pd.read_csv(stim1.replace('jpg', 'txt'), header=None, skiprows=1, names=['idx', 'posX', 'posY', 'defX', 'defY'])
    stim2_dfm = pd.read_csv(stim2.replace('jpg', 'txt'), header=None, skiprows=1, names=['idx', 'posX', 'posY', 'defX', 'defY'])

    for au in stim1_au_data.columns:
        # get AU probability estimate
        stim1_score = stim1_au_data[au].iloc[0]
        stim2_score = stim2_au_data[au].iloc[0]

        # the image with higher estimate is defined as the model's response 
        if stim1_score > stim2_score or stim1_score == stim2_score:
            stim1_dfm['au'], stim1_dfm['stim_order'], stim1_dfm['stim'], stim1_dfm['score'], stim1_dfm['response'] = au, 0, stim1, stim1_score, True
            stim2_dfm['au'], stim2_dfm['stim_order'], stim2_dfm['stim'], stim2_dfm['score'], stim2_dfm['response'] = au, 1, stim2, stim2_score, False 
        elif stim1_score < stim2_score: 
            stim1_dfm['au'], stim1_dfm['stim_order'], stim1_dfm['stim'], stim1_dfm['score'], stim1_dfm['response'] = au, 0, stim1, stim1_score, False
            stim2_dfm['au'], stim2_dfm['stim_order'], stim2_dfm['stim'], stim2_dfm['score'], stim2_dfm['response'] = au, 1, stim2, stim2_score, True
            
        df_au = pd.concat([stim1_dfm, stim2_dfm])
        all_dfs.append(df_au)

In [ ]:
df = pd.concat(all_dfs)
df.to_csv('./outputs/londonset_output/070.csv', index=False)

In [ ]:
df

# Average dfms of all chosen stimuli

In [ ]:
# landmark_locs = {
#     'silhouette': [
#         10,  338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288,
#         397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136,
#         172, 58,  132, 93,  234, 127, 162, 21,  54,  103, 67,  109
#     ],
    
#     'lipsUpperOuter': [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291],
#     'lipsLowerOuter': [146, 91, 181, 84, 17, 314, 405, 321, 375, 291],
#     'lipsUpperInner': [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308],
#     'lipsLowerInner': [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308],

#     'lips': [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308],
    
#     'rightEyeUpper0': [246, 161, 160, 159, 158, 157, 173],
#     'rightEyeLower0': [33, 7, 163, 144, 145, 153, 154, 155, 133],
#     'rightEyeUpper1': [247, 30, 29, 27, 28, 56, 190],
#     'rightEyeLower1': [130, 25, 110, 24, 23, 22, 26, 112, 243],
#     'rightEyeUpper2': [113, 225, 224, 223, 222, 221, 189],
#     'rightEyeLower2': [226, 31, 228, 229, 230, 231, 232, 233, 244],
#     'rightEyeLower3': [143, 111, 117, 118, 119, 120, 121, 128, 245],
    
#     'rightEyebrowUpper': [156, 70, 63, 105, 66, 107, 55, 193],
#     'rightEyebrowLower': [35, 124, 46, 53, 52, 65],
    
#     'rightEyeIris': [473, 474, 475, 476, 477],
    
#     'leftEyeUpper0': [466, 388, 387, 386, 385, 384, 398],
#     'leftEyeLower0': [263, 249, 390, 373, 374, 380, 381, 382, 362],
#     'leftEyeUpper1': [467, 260, 259, 257, 258, 286, 414],
#     'leftEyeLower1': [359, 255, 339, 254, 253, 252, 256, 341, 463],
#     'leftEyeUpper2': [342, 445, 444, 443, 442, 441, 413],
#     'leftEyeLower2': [446, 261, 448, 449, 450, 451, 452, 453, 464],
#     'leftEyeLower3': [372, 340, 346, 347, 348, 349, 350, 357, 465],
    
#     'leftEyebrowUpper': [383, 300, 293, 334, 296, 336, 285, 417],
#     'leftEyebrowLower': [265, 353, 276, 283, 282, 295],
    
#     'leftEyeIris': [468, 469, 470, 471, 472],
    
#     'midwayBetweenEyes': [168],
    
#     'noseTip': [1],
#     'noseBottom': [2],
#     'noseRightCorner': [98],
#     'noseLeftCorner': [327],
    
#     'rightCheek': [205],
#     'leftCheek': [425],

#     'cheek': [205, 425]
# }

In [ ]:
# img_file = "./images/londonset_scaled/001.jpg"
# config_file = "./configs/mediapipe.toml"

In [ ]:
# AU = 'AU12'
# only_significant_landmarks = False

# df = pd.read_csv('./outputs/londonset_output/001.csv').reset_index(drop=True)

# df_au = df.loc[df.au == AU]
# df_au

In [ ]:
# def extract_single_kernel(data_df, feature_id = 'feature', value_id = 'value', response_id = 'response'):
#     feature_average = data_df.groupby([feature_id,response_id])[value_id].mean().reset_index()
#     positives = feature_average.loc[feature_average[response_id] == True].reset_index()
#     negatives = feature_average.loc[feature_average[response_id] == False].reset_index()
#     kernels = pd.merge(positives, negatives, on=feature_id, suffixes=('_true','_false'))
#     kernels['kernel_value'] = kernels['%s_true'%value_id] - kernels['%s_false'%value_id]
#     kernels = kernels[[feature_id,'kernel_value']].set_index(feature_id)
#     kernels.index.names = ['feature']
    
#     return kernels


# def normalize_kernel(kernel):
#     rms = np.sqrt((kernel.kernel_value**2).mean())
#     normalized_kernel = kernel.copy()
#     normalized_kernel['kernel_value'] = normalized_kernel['kernel_value'].apply(lambda x: x/rms)
        
#     return normalized_kernel


# def compute_kernel(data, normalize=True):
#     kernel_x = extract_single_kernel(
#         data,
#         feature_id = 'idx',
#         value_id = 'defX',
#         response_id = 'response'
#     )   
#     kernel_y = extract_single_kernel(
#         data,
#         feature_id = 'idx',
#         value_id = 'defY',
#         response_id = 'response'
#     )

#     if normalize:
#         kernel_x = normalize_kernel(kernel_x)
#         kernel_y = normalize_kernel(kernel_y)

#     kernel = pd.DataFrame({
#         'index': kernel_x.index,
#         'posX': data.posX[:len(kernel_x.index)],
#         'posY': data.posY[:len(kernel_x.index)],
#         'defX': kernel_x.kernel_value.values,
#         'defY': kernel_y.kernel_value.values
#     }).reset_index(drop=True)
    
#     return kernel       

In [ ]:
# def save_dfmxy(filename, kernel):
#     # formatting for saving as dfmxy
#     txt_rows = []
#     for el in kernel.to_string(index=False, index_names=False).split('\n'):
#         txt_rows.append(",".join(el.split()))
    
#     # write to text file
#     with open(filename, 'a') as f:
#         for row in txt_rows:
#             f.write(row+"\n")


# def show_transform(dfmxy_file, kernel, scale):
#     kernel = kernel.copy()
    
#     # rescale
#     kernel.defX = scale * kernel.defX
#     kernel.defY = scale * kernel.defY
#     kernel.set_index('index').to_csv(dfmxy_file, index_label = 'index', header=None)
    
#     dfmxy = FaceWarp.load_dfmxy(dfmxy_file)
#     transformed = cleese.process_file(
#                                         FaceWarp,
#                                         img_file,
#                                         config_file,
#                                         dfmxy=dfmxy
#                                     )
#     original_img = Image.open(img_file)
#     transformed_img = Image.fromarray(transformed)
#     # diff_img = Image.fromarray(np.asarray(original_img) - transformed)
#     transformed_img.save(dfmxy_file.replace('dfmxy', 'jpg'))
    
#     fig, axs = plt.subplots(ncols=2)
#     axs[0].imshow(original_img)
#     axs[1].imshow(transformed_img, zorder=0)
#     # axs[2].imshow(diff_img, zorder=0)
#     axs[0].axis('off')
#     axs[1].axis('off')
#     # axs[2].axis('off')

In [ ]:
# if only_significant_landmarks:
#     # test = df_au.groupby('idx').apply(lambda x: pd.Series({
#     #     'ttest': pg.ttest(
#     #         np.sqrt(np.square(x.loc[x.response==True].defX) + np.square(x.loc[x.response==True].defY)),
#     #         np.sqrt(np.square(x.loc[x.response==False].defX) + np.square(x.loc[x.response==False].defY)),
#     #         paired=True
#     #     )['p-val'].iloc[0]
#     # }), include_groups=False)
#     # sig = test.loc[test.ttest < 0.05].reset_index()
#     data = df_au.loc[df_au.idx.isin(monalisa.iloc[:, 0])]
# else:
#     data = df_au

# kernel = compute_kernel(data, normalize=True)
# save_dfmxy(f'./kernels/{os.path.splitext(os.path.basename(img_file))[0]}_{AU}.dfmxy', kernel)
# show_transform(f'./kernels/{os.path.splitext(os.path.basename(img_file))[0]}_{AU}.dfmxy', kernel, 3)

#### Average (defX, defY) of all kernels

In [ ]:
# monalisa = pd.read_csv('./monalisa.random.dfmxy')

In [ ]:
# df = pd.read_csv('./kernels/001_AU12.dfmxy', header=None)
# df

In [ ]:
# def average_kernels(au):
#     kernels = []
#     for f in glob.glob(f'./kernels/*_{au}.dfmxy'):
#         df = pd.read_csv(f, header=None)
#         kernels.append(df.to_numpy())

#     avg = np.mean(kernels, axis=0)
#     avg_kernel = pd.DataFrame({
#         'index': avg[:, 0],
#         'posX': avg[:, 1],
#         'posY': avg[:, 2],
#         'defX': avg[:, 3],
#         'defY': avg[:, 4]
#     })

#     return avg_kernel

In [ ]:
# avgk = average_kernels('AU12')
# avgk

In [ ]:
# img_file = "./images/londonset/004.jpg"
# config_file = "./configs/mediapipe.toml"

In [ ]:
# show_transform(f'./kernels/avgkernel_AU12.dfmxy', avgk, 2)

# Extras

In [ ]:
# k1 = pd.read_csv('./AM04NES_AU12.dfmxy', header=None)
# k1.columns = ['idx', 'posX', 'posY', 'defX', 'defY']

# k2 = pd.read_csv('./AF01NES_AU12.dfmxy', header=None)
# k2.columns = ['idx', 'posX', 'posY', 'defX', 'defY']

In [ ]:
# commons = list(set(k1['idx'].to_list()) & set(k2['idx'].to_list()))
# k1 = k1.loc[k1['idx'].isin(commons)].reset_index(drop=True)
# k2 = k2.loc[k2['idx'].isin(commons)].reset_index(drop=True)

In [ ]:
# k = k1.merge(k2, how='left', on='idx', suffixes=(None, '_other'))
# k

In [ ]:
# k['avgX'] = k.apply(lambda x: (x.defX + x.defX_other)/2, axis=1)
# k['avgY'] = k.apply(lambda x: (x.defY + x.defY_other)/2, axis=1)
# k

In [ ]:
# k_alt1 = k[['idx', 'posX', 'posY', 'avgX', 'avgY']].rename(columns={'idx': 'index', 'avgX': 'defX', 'avgY': 'defY'})
# k_alt2 = k[['idx', 'posX_other', 'posY_other', 'avgX', 'avgY']].rename(columns={'idx': 'index', 'posX_other': 'posX', 'posY_other': 'posY', 'avgX': 'defX', 'avgY': 'defY'})

In [ ]:
# show_transform('./test.dfmxy', k_alt1, 5)

In [ ]:
# for i in monalisa.iloc[:, 0]:
#     for el in landmark_locs:
#         if i in landmark_locs[el]:
#             print(f'{i}: {el}')

In [ ]:
info = pd.read_csv('./images/londonset_scaled/london_faces_info.csv')
info

In [ ]:
info.loc[info.face_eth != 'white'].reset_index(drop=True)